# Trying Open Source models through Groq

In [12]:
import os
from dotenv import load_dotenv
from rich import print
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [13]:
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Translate the following text from English to Urdu"),
    HumanMessage(content="Every individual has some unique talent"),
]
messages

[SystemMessage(content='Translate the following text from English to Urdu'),
 HumanMessage(content='Every individual has some unique talent')]

In [14]:
llm = ChatGroq(model="gemma2-9b-it")
response = llm.invoke(messages)

print(response)

AIMessage(
    content="ہر فرد میں کوئی نہ کوئی خاص صلاحیت ہوتی ہے۔ \n\n\nLet me know if you'd like me to translate anything 
else!\n",
    response_metadata={
        'token_usage': {
            'completion_tokens': 33,
            'prompt_tokens': 28,
            'total_tokens': 61,
            'completion_time': 0.066,
            'prompt_time': 0.002721916,
            'queue_time': None,
            'total_time': 0.06872191600000001
        },
        'model_name': 'gemma2-9b-it',
        'system_fingerprint': 'fp_10c08bf97d',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-f1cf5d3e-26a5-42ed-a80e-67e252065acd-0',
    usage_metadata={'input_tokens': 28, 'output_tokens': 33, 'total_tokens': 61}
)

In [15]:
from langchain_core.output_parsers import StrOutputParser
# using output parser to get the actual output from response
parser = StrOutputParser()
print(parser.invoke(response))

ہر فرد میں کوئی نہ کوئی خاص صلاحیت ہوتی ہے۔ 


Let me know if you'd like me to translate anything else!

## Chaining components using LCEL

In [16]:
chain =  llm | parser
chain.invoke(messages)

'ہر فرد میں کوئی نہ کوئی خاص صلاحیت ہوتی ہے۔ \n'

In [19]:
from langchain_core.prompts import ChatPromptTemplate

template = "Translate the following text into {language}"

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("user", "{text}")
])

prompt

ChatPromptTemplate(input_variables=['language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translate the following text into {language}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [21]:
# if we invoke this prompt, it will populate the input variables and create a list of
# messages (System and Human message) just like we were previously doing
prompt.invoke({
    "language": "Arabic",
    "text": "Nothing is easy in this world if you don't try."
})  # if we store prompt in a variable and apply 'to_messages' method on it, it will return just messages

ChatPromptValue(messages=[SystemMessage(content='Translate the following text into Arabic'), HumanMessage(content="Nothing is easy in this world if you don't try.")])

In [22]:
chain = prompt | llm | parser

chain.invoke({
    "language": "Arabic",
    "text": "Nothing is easy in this world if you don't try."
})

"لا شيء سهل في هذا العالم إلا إذا حاولت. \n\n\n(La shey' sahil fi hadha al-ʿālam illā idhā ḥāwalta.)\n"